# Домашнее задание, Игнатьев Д.

## Функция для подсчета PMI

In [ ]:
!pip install razdel
!pip install pymorphy2
!pip install string
import nltk
nltk.download("stopwords")

In [3]:
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import numpy as np
import re
from string import punctuation
from nltk.corpus import stopwords

stops = set(stopwords.words('russian') + ["это", "весь"])
morph = MorphAnalyzer()

def normalize(text):
    tokens = re.findall('[а-яёa-z0-9]+', text.lower())
    normalized_text = [morph.parse(word)[0].normal_form for word \
                                                            in tokens]
    normalized_text = [word for word in normalized_text if len(word) > 2 and word not in stops]
    
    return normalized_text

def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

In [4]:
import math

In [5]:
import gensim

In [6]:
dvach = open('2ch_corpus.txt', encoding="UTF-8").read()
news = open('lenta.txt', encoding="UTF-8").read()

In [ ]:
news_corpus = preprocess(news)

In [ ]:
def PMI_scorer(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count):
    if bigram_count < min_count:
        return 0
    try:
        bigram_prob = bigram_count / corpus_word_count
        worda_prob = worda_count / corpus_word_count
        wordb_prob = wordb_count / corpus_word_count
        score = math.log(bigram_prob / (worda_prob * wordb_prob)) 
    except ZeroDivisionError:
        return 0
    return score

In [ ]:
ph = gensim.models.Phrases(news_corpus, threshold=0.01, scoring=PMI_scorer)
p = gensim.models.phrases.Phraser(ph)

In [ ]:
ph2 = gensim.models.Phrases(p[news_corpus])
p2 = gensim.models.phrases.Phraser(ph2)

In [ ]:
p2[p[news_corpus[330]]]

['настоящий_время',
 'стационар',
 'больница_город',
 'находиться',
 'раненый',
 'критический_состояние',
 'трое',
 'обжечь',
 'процент',
 'кожа']

In [ ]:
p2[p[news_corpus[545]]]

['общий_сумма',
 'долг',
 'девять_тысяча',
 'работник',
 'образование',
 'республика',
 'зарплата',
 'выплата',
 'сентябрь',
 'составить_миллион_рубль']

## Триграммная модель

In [7]:
from nltk.tokenize import sent_tokenize

In [8]:
def normalize_ngrams(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
sentences_dvach = [['<start>', '<start>'] + normalize_ngrams(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>', '<start>'] + normalize_ngrams(text) + ['<end>'] for text in sent_tokenize(news)]

In [12]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [13]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))


unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))

In [14]:
from scipy.sparse import dok_matrix

In [15]:
matrix_dvach = dok_matrix((len(bigrams_dvach), 
                   len(unigrams_dvach)), dtype=np.float32)
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {bigram:i for i, bigram in enumerate(id2bigram_dvach)}

for ngram in trigrams_dvach:
    split = ngram.split(" ")
    bigram = split[0] + " " + split[1]
    word = split[2]
    matrix_dvach[bigram2id_dvach[bigram], word2id_dvach[word]] = (trigrams_dvach[ngram]/
                                                                     bigrams_dvach[bigram])
    # [bigram2id_dvach[bigram]][word2id_dvach[word]]

In [ ]:
matrix_dvach.get_shape()

(1032321, 189517)

In [20]:
# создадим матрицу вероятностей перейти из 1 слов в другое
matrix_news = dok_matrix((len(bigrams_news), 
                   len(unigrams_news)), dtype=np.float32)
id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {bigram:i for i, bigram in enumerate(id2bigram_news)}

# вероятность расчитываем точно также
for ngram in trigrams_news:
    split = ngram.split(" ")
    bigram = split[0] + " " + split[1]
    word = split[2]
    matrix_news[bigram2id_news[bigram], word2id_news[word]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigram])

In [16]:

def generate(matrix, id2word, id2bigram, word2id, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    shape = matrix.get_shape()
    
    for i in range(n):
        chosen = np.random.choice(shape[1], p=matrix.getrow(current_idx).toarray().flatten())
        # chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        new_word = id2word[chosen]
        text.append(id2word[chosen])
        
        bigram = id2bigram[current_idx]
        next_bigram = bigram[bigram.find(" ") + 1:] + " " + new_word
        
        if new_word == '<end>':
            current_idx = bigram2id['<start> <start>']
        else:
            current_idx = bigram2id[next_bigram]
    return ' '.join(text)

### Новые тексты

In [17]:
print(generate(matrix_dvach, id2word_dvach, id2bigram_dvach, word2id_dvach, bigram2id_dvach).replace('<end>', '\n'))

я тоже в деревни как бы надо выкидывать мультик нахуй все 
 аккаунт тоже с малолетства зависала на чатурбейтах шлюховала и описывался парад проституток 
 и ты все еще здесь собираю картиночки и комиксы делюсь с вами каши не сваришь но обезвоживание мне теперь ничего не производится ничего а её батя продать запись на этот счёт 
 если у тебя не пидорнули фантазер 
 узрите же как видишь о найс мерки мои убили 
 бред 28 48 но их ничтожно мало 
 это потому что она вам и доказательство что пиздоглазые не могут попасть летающие снаряды по этой фотке не


In [18]:
del matrix_dvach

In [22]:
print(generate(matrix_news, id2word_news, id2bigram_news, word2id_news, bigram2id_news).replace('<end>', '\n'))

планируется что в вопросе ликвидации сторон 
 национальная ассоциация телевещателей и журнал маркетинг и маркетинговые исследования в первый раз это является нарушением закона если оно взяло на себя ответственность за другие террористические акции на мавзолее осмоловский позвонил в телестудию секретный протокол встречи представителей службы безопасности проверяющих проводили в эти города будут опрыскиваться и дальше продолжать переговоры с министерством связи и так далее 
 про самое важное исключить возможность того что происходит на фоне продолжения борьбы за контроль над каналом в который войдут губернаторы и руководители американского eximbank нанедавней встрече в кремле говорится в документе по мнению руководства хорошо владеющий ситуацией


In [23]:
del matrix_news

### Старые тексты

In [ ]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach).replace('<end>', '\n'))

они потеряли со всеми 
 молодец 
 а остальные 4 имя той толпы следовательно вставлять как только стареть и фаерфаксе свг спрайт 
 и пропустить что это чтоб задыхалась так будь ты хуй на 10 назад 
 от 20 черепов – я посмотрю как худая смотрит 
 с четвёртого энергоблока 
 одним выстрелом расхуярить чтобы было видно что совершил преступление перед прыжком скажу только малая зона в дни месяцы слабо подумать что была мечта казуала 
 с крайностями 
 блин сделали потому что он спит а не дна хуже чем китай 
 год поставил 
 св 
 геноцида ксеносов в


In [ ]:
print(generate(matrix_news, id2word_news, word2id_news).replace('<end>', '\n'))

иначе представители таких как сми сказал генерал колонна из юар нельсон автор сюжета было совершено покушение на своих наблюдателей передает риа новости 
 по полному расчету с первым шагом на 15 сентября 
 на средства буквально за рубежом 
 полностью поддержала точку зрения на внешних врагов то будут очень похож на политический советник президента около ста семей 
 владелец предприятия 
 касаясь ситуации в длинном перечне 85 лет и оружия был обнаружен тайник с 1993 по умолчанию не поможет ведь никто не имеет самые решительные действия в пресс-службе всемирного банка 
 в порту сочи началась эпидемия гриппа достигнет рекордного уровня


### Наблюдение:
после добавления триграмм последовательности слов в тексте в целом стали более осмысленными. Например,
"исключить возможность того что происходит" или "это является нарушением закона".